In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [4]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [5]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female



In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [9]:
inputs.columns[inputs.isna().any()]


Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]


0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()


,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)


In [13]:
len(X_train)

712

In [14]:
len(X_test)

179

In [19]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [20]:
model.fit(X_train,y_train)

GaussianNB()

In [21]:
model.score(X_test,y_test)


0.7835820895522388

In [22]:
X_test[0:10]

,Pclass,Age,Fare,female
799,3,30.0,24.1500,1
307,1,17.0,108.9000,1
387,2,36.0,13.0000,1
658,2,23.0,13.0000,0
21,2,34.0,13.0000,0
726,2,30.0,21.0000,1
232,2,59.0,13.5000,0
771,3,48.0,7.8542,0
833,3,23.0,7.8542,0
762,3,20.0,7.2292,0


In [23]:
y_test[0:10]

799    0
307    1
387    1
658    0
21     1
726    1
232    0
771    0
833    0
762    1
Name: Survived, dtype: int64

In [24]:
model.predict(X_test[0:10])

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [25]:
model.predict_proba(X_test[:10])

array([[0.38196903, 0.61803097],
       [0.00262758, 0.99737242],
       [0.22658884, 0.77341116],
       [0.91174326, 0.08825674],
       [0.92028048, 0.07971952],
       [0.21762228, 0.78237772],
       [0.90186402, 0.09813598],
       [0.96114787, 0.03885213],
       [0.95862556, 0.04137444],
       [0.95637418, 0.04362582]])

Calculate the score using cross validation

In [28]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=10)

array([0.80952381, 0.73015873, 0.68253968, 0.79032258, 0.88709677,
       0.69354839, 0.77419355, 0.74193548, 0.83870968, 0.79032258])